## Latent Dimention - First Exploration

Con la arquitectura de nuestro CAE determinada en 4.0, buscamos la dimensión mínima del espacio latente que pueda reproducir los datos de entrada con una dada precisión. Para eso vamos a entrenar distintos modelos en los que solo se varía la dimensión del espacio latente y buscar un codo en el score medio de las últimas 10 epochs.

In [ ]:
import json
import os
import pickle

import pandas as pd
import tensorflow as tf
from tqdm import tqdm

from deep_scattering_models.data.create_data import load_data
from deep_scattering_models.models.model_wrappers import build_cae_architecture
from deep_scattering_models.models.select_model import load_configuration, k_fold_cv, save_configuration
from deep_scattering_models.visualization.visualize import plot_polarization_signature, plot_history

Comenzamos cargando los datos y la configuración de nuestro CAE

In [ ]:
train_set = load_data("processed/spm_signatures_r_scale_train_set")
test_set = load_data("processed/spm_signatures_r_scale_test_set")

cae_configuration = load_configuration("model_architecture_spm.json")

Variamos la dimensión del espacio latente de 16 unidades a 1 unidad. Registramos el MSE y el MAE del conjunto de testeo para cada una de las configuraciones.

In [ ]:
# Initialize list for results
models_scores = []
not_tested_configurations = []

for lat_dim in tqdm(range(1, 17)):
    
    cae_configuration.update({"latent_dimension" : lat_dim})

    try:
        k_fold_score = k_fold_cv(
            train_set, 
            build_cae_architecture, 
            cae_configuration
            )
    
    except tf.errors.ResourceExhaustedError:
        # If gpu memory is exhausted
        not_tested_configurations.append(cae_configuration)
        
        raise Warning(('La actual configuración agotó los recursos '
        f'de memoria y no fue evaluada: \n {cae_configuration}'))  
    
    else:
        # Update scores
        cae_configuration.update(k_fold_score)
        models_scores.append(cae_configuration)

    finally:
        continue    

In [ ]:
df_score = pd.DataFrame.from_records(models_scores)
df_score

Guardamos los scores serializados en un pkl

In [ ]:
# Get resutls directory path
src_dir = os.path.normpath(os.getcwd() + '/../..')
results_dir = os.path.join(src_dir, 'results/spm')

# File name and dir
file_name = 'spm_ld_scores.pkl'
file_dir = os.path.join(results_dir, file_name)

# Save model_scores as pkl
with open(file_dir, 'wb') as f:
    pickle.dump(models_scores, f)